In [28]:
import json
 
with open('/kaggle/input/legalner-dataset/NER_TRAIN_PREAMBLE.json','r') as f:
    data = json.load(f)

In [29]:
training_data = {'classes' : [], 'annotations' : []}
for example in data:
  
  if len(example['annotations']) >= 1 and len(example['annotations'][0]['result']) >= 1:
    temp_dict = {}
    temp_dict['text'] = example['data']['text']
    temp_dict['entities'] = []
    start = example['annotations'][0]['result'][0]['value']['start']
    end = example['annotations'][0]['result'][0]['value']['end']
    label = example['annotations'][0]['result'][0]['value']['labels'][0].upper()
    temp_dict['entities'].append((start, end, label))
    training_data['annotations'].append(temp_dict)
print(training_data['annotations'][0])
#     for train_data in example['annotations'][0]['result']:
#         start = train_data['value']['start']
#         end = train_data['value']['end']
#         label = train_data['value']['labels'][0].upper()
#         temp_dict['entities'].append((start, end, label))
#         training_data['annotations'].append(temp_dict)
        


{'text': "In The High Court Of Kerala At Ernakulam\n\nCrl Mc No. 1622 of 2006()\n\n\n1. T.R.Ajayan, S/O. O.Raman,\n                      ...  Petitioner\n\n                        Vs\n\n\n\n1. M.Ravindran,\n                       ...       Respondent\n\n2. Mrs. Nirmala Dinesh, W/O. Dinesh,\n\n                For Petitioner  :Sri.A.Kumar\n\n                For Respondent  :Smt.M.K.Pushpalatha\n\nThe Hon'ble Mr. Justice P.R.Raman\nThe Hon'ble Mr. Justice V.K.Mohanan\n\n Dated :07/01/2008\n\n O R D E R\n", 'entities': [(7, 40, 'COURT')]}


In [30]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [31]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("trn_data.spacy") # save the docbin object

100%|██████████| 1558/1558 [00:03<00:00, 464.13it/s]


In [33]:
!python -m spacy init fill-config /kaggle/input/configfiles/base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [35]:
!python -m spacy train config.cfg --output ./ --paths.train ./trn_data.spacy --paths.dev ./trn_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-10 18:48:05,644] [INFO] Set up nlp object from config
[2022-12-10 18:48:05,655] [INFO] Pipeline: ['transformer', 'ner']
[2022-12-10 18:48:05,660] [INFO] Created vocabulary
[2022-12-10 18:48:05,661] [INFO] Finished initializing nlp object
Downloading: 100%|██████████████████████████████| 480/480 [00:00<00:00, 393kB/s]
Downloading: 100%|████████████████████████████| 878k/878k [00:01<00:00, 898kB/s]
Downloading: 100%|████████████████████████████| 446k/446k [00:00<00:00, 461kB/s]
Downloading: 100%|█████████████████████████| 1.29M/1.29M [00:01<00:00, 1.13MB/s]
Downloading: 100%|███████████████████████████| 316M/316M [00:21<00:00, 15.5MB/s]
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias',